# 获取课程表
## 导入相关库

In [10]:
import requests 
import json
import re
from bs4 import BeautifulSoup

## 预先操作
1. 加密密码

In [4]:
def get_enc_password(origin_pwd: str) -> str:
    ENC_URL = 'http://cas.ecjtu.edu.cn/cas/loginPasswdEnc'
    enc_response = requests.post(ENC_URL, data={'pwd': origin_pwd})

    _ = enc_response.content.decode('utf8').replace("'", '"')
    return json.loads(_)['passwordEnc']

password = "your_password"
print(get_enc_password(password))

30052603102467EDE4DA3F7C61F023727039121D4EC611A63A537C2277D34897F693CC5142911946DFDD34206A55049E


2. 收集相关的api

In [5]:
ecjtu_login = "http://cas.ecjtu.edu.cn/cas/login" # 智慧交大登录页
jwxt_login = "https://jwxt.ecjtu.edu.cn/stuMag/Login_dcpLogin.action" # 教务系统登录页
ecjtu_to_jwxt = "http://cas.ecjtu.edu.cn/cas/login?service=https%3A%2F%2Fjwxt.ecjtu.edu.cn%2FstuMag%2FLogin_dcpLogin.action" #智慧交大登录教务系统
get_classes = "https://jwxt.ecjtu.edu.cn/Schedule/Weekcalendar_getTodayWeekcalendar.action" # 获取课程信息(post)
get_grade = "https://jwxt.ecjtu.edu.cn/scoreQuery/stuScoreQue_getStuScore.action?item=0401" # 获取成绩信息(get)

## 登录

In [6]:
def login(username:str,password:str,session:requests.Session) -> requests.Session:
    # 关闭警告
    requests.packages.urllib3.disable_warnings()
    # 获取密码
    enc = get_enc_password(password)
    # 打包登录body
    login_data = {
        'username': username,
        'password': enc,
        'lt': '',
        "service":"http://portal.ecjtu.edu.cn/dcp/index.jsp",
    }
    # 获取lt
    headers = {
        # 设置UA
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        # 设置Host
        "Host": "cas.ecjtu.edu.cn"
    }
    response = session.get(ecjtu_login,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    login_data['lt'] = soup.find('input', {'name': 'lt'})['value']
    # 登录
    headers_append = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Referer": "http://cas.ecjtu.edu.cn/cas/login"
      }
    headers.update(headers_append)
    response = session.post(ecjtu_login, data=login_data,headers=headers,allow_redirects=False)
    if not response.cookies.get('CASTGC'):
        print("账户或密码错误")
        return None
    # 获取教务JSESSIONID
    session.get(jwxt_login,headers=headers,allow_redirects=False,verify=False)
    # 获取教务跳转登录链接
    response_url = session.get(ecjtu_to_jwxt,allow_redirects=False)
    # 登录教务系统
    result = session.get(response_url.headers['Location'])
    if result.status_code != 200:
        print("教务系统错误，登录失败")
        return None
    print("登录成功")
    return session

session = requests.Session()
login("2022211003000111","mzdwzyddn0",session)
classes = session.post(get_classes,data={"date":"2024-04-05"})
for k,v in classes.json().items():
    if k == "weekcalendarpojoList":
        for item in v:
            for i,j in item.items():
                print(i," ",j)
            print("\n")


登录成功
classString   1-2
courseRequire   必修课
teacherName   吕敬钦
weekDay   5
classRoom   31-627
classSpan   1,2
course   软件测试技术
className   软件测试技术(20232-2)
weekSpan   1-16
pkType   上课


classString   3-4
courseRequire   必修课
teacherName   叶云青
weekDay   5
classRoom   31-625
classSpan   3,4
course   计算机网络
className   计算机网络(20232-4)
weekSpan   1-16
pkType   上课




## 获取成绩情况

In [7]:
session = requests.Session()
login("2022211003000111","mzdwzyddn0",session)
grade_html = session.get(get_grade)
print(grade_html.text)

登录成功



<!DOCTYPE html>
<html lang="zh-cn">
  <head>
    <meta charset="utf-8"/>
	<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
	<meta name="renderer" content="webkit">
    <base href="https://jwxt.ecjtu.edu.cn/">
    <title>华东交通大学教务综合管理信息系统</title>
	<link rel="icon" type="image/vnd.microsoft.icon" href="images/ecjtu_logo.ico" />   
	<meta http-equiv="keywords" content="华东交通大学,教务管理系统,学生成绩,胡振立,刘锦清,王海宝,朱济世">
	<meta http-equiv="description" content="华东交通大学教务管理系统-学生成绩">		
	<script src="scripts/jquery-1.11.1.min.js"></script> 
    <script src="scripts/edumis_function.js"></script> 
	<link rel="stylesheet" type="text/css" href="styles/edumis.css">
    <link rel="stylesheet" type="text/css" href="styles/main.css">
    <link rel="stylesheet" type="text/css" href="styles/scorequery.css" />
</head>
<body>
<!-- header begin-->

<div class="header">
	<div id="head-center">
		<a href="" title="首页"><div id="logo-edumis"></div></a>
	    <div id="info">
			<ul class="nav-item">
				<l

### 绩点情况

In [8]:
def get_grade_count(grade_html:requests.Response) -> dict:
    soup = BeautifulSoup(grade_html.text, 'html.parser')
    data_row = soup.find_all("tr")[3]
    data = [td.text for td in data_row.find_all("td")]
    grade_count = {
        "姓名":data[1],
        "班级":data[2],
        "平均学分绩点":data[6],
    }
    return grade_count
print(get_grade_count(grade_html))

{'姓名': '刘鑫林', '班级': '正常|专业调整|有学籍', '平均学分绩点': '3.46'}


## 成绩情况（按学期）

In [11]:

def get_grade_from_semester(semester:str,grade_html:requests.Response) -> list:
    tsemester = semester.replace(".","_")
    grade_from_semester_list = []
    soup = BeautifulSoup(grade_html.text, 'html.parser')

    pattern = re.compile(r'\b{}\b'.format(tsemester))
    ul_tags = soup.find_all("ul",class_=pattern)
    for ul_tag in ul_tags:
        li_list = ul_tag.find_all("li")
        li_values = [li.text for li in li_list]
        grade_from_semester = {
            "课程学期":semester,
            "课程":{
                "课程名":li_values[1],
                "课程性质":li_values[2],
                "学分":li_values[4],
                "成绩":li_values[5],
            },
        }
        grade_from_semester_list.append(grade_from_semester)
    return grade_from_semester_list

print(get_grade_from_semester("2022.1",grade_html))

[{'课程学期': '2022.1', '课程': {'课程名': '【1500100101】职业生涯与发展规划', '课程性质': '必修课', '学分': '0.5', '成绩': '良好'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1500190090】专业导论', '课程性质': '必修课', '学分': '0', '成绩': '合格'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1500190200】军事技能', '课程性质': '必修课', '学分': '1', '成绩': '合格'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1505100031】体育Ⅰ', '课程性质': '必修课', '学分': '1', '成绩': '85'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1505101460】国家安全与军事理论', '课程性质': '必修课', '学分': '2', '成绩': '良好'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1508100011】高等数学(A)Ⅰ', '课程性质': '必修课', '学分': '6', '成绩': '82'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1508100201】土建工程制图Ⅰ', '课程性质': '必修课', '学分': '3', '成绩': '83'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1509100011】大学英语Ⅰ', '课程性质': '必修课', '学分': '3', '成绩': '84'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1514100151】形势与政策Ⅰ', '课程性质': '必修课', '学分': '0.5', '成绩': '良好'}}, {'课程学期': '2022.1', '课程': {'课程名': '【1514100170】思想道德与法治', '课程性质': '必修课', '学分': '3', '成绩': '87'}}, {'课程学期': '2022.1', '课程': {'课程名': '【15211012